In [ ]:
# install the package
%pip install --upgrade mani_skill
# install a version of torch that is compatible with your system
%pip install torch torchvision torchaudio

In [ ]:
#MP_SOLUTIONS = {
#    "PickCube-v1": solvePickCube,
#    "StackCube-v1": solveStackCube,
#    "PegInsertionSide-v1": solvePegInsertionSide,
#    "PlugCharger-v1": solvePlugCharger,
#    "PushCube-v1": solvePushCube,
#}

  parser.add_argument("-e", "--env-id", type=str, default="PickCube-v1", help=f"Environment to run motion planning solver on. Available options are {list(MP_SOLUTIONS.keys())}")
    parser.add_argument("-o", "--obs-mode", type=str, default="none", help="Observation mode to use. Usually this is kept as 'none' as observations are not necesary to be stored, they can be replayed later via the mani_skill.trajectory.replay_trajectory script.")
    parser.add_argument("-n", "--num-traj", type=int, default=10, help="Number of trajectories to generate.")
    parser.add_argument("--only-count-success", action="store_true", help="If true, generates trajectories until num_traj of them are successful and only saves the successful trajectories/videos")
    parser.add_argument("--reward-mode", type=str)
    parser.add_argument("-b", "--sim-backend", type=str, default="auto", help="Which simulation backend to use. Can be 'auto', 'cpu', 'gpu'")
    parser.add_argument("--render-mode", type=str, default="rgb_array", help="can be 'sensors' or 'rgb_array' which only affect what is saved to videos")
    parser.add_argument("--vis", action="store_true", help="whether or not to open a GUI to visualize the solution live")
    parser.add_argument("--save-video", action="store_true", help="whether or not to save videos locally")
    parser.add_argument("--traj-name", type=str, help="The name of the trajectory .h5 file that will be created.")
    parser.add_argument("--shader", default="default", type=str, help="Change shader used for rendering. Default is 'default' which is very fast. Can also be 'rt' for ray tracing and generating photo-realistic renders. Can also be 'rt-fast' for a faster but lower quality ray-traced renderer")
    parser.add_argument("--record-dir", type=str, default="demos", help="where to save the recorded trajectories")

In [ ]:
!python -m mani_skill.examples.motionplanning.panda.run -e "PickCube-v1"\
    --save-video \
    --num-traj 10 \
    --only-count-success \
    --record-dir /content/drive/MyDrive/Data/Training/generated
    



In [ ]:

file_path = "/content/drive/MyDrive/Data/Training/generated"

In [ ]:

!python -m mani_skill.trajectory.replay_trajectory \
  --traj-path /content/drive/MyDrive/Data/Training/generated \
  --save-traj \
  --obs-mode pointcloud \
  --sim-backend gpu